# RAG

A continuación se muestra una descripción general de alto nivel del sistema que queremos construir:


<img src='images/img_1.png' width="800">

# PARTE I

Empecemos cargando las variables de entorno que necesitamos utilizar.

## Setting up the model
Definamos el modelo LLM que utilizaremos como parte del flujo de trabajo.

In [ ]:
import os
from dotenv import load_dotenv


load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Este es el video de YouTube que vamos a utilizar.
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=qcJM0bM3D0Q'"

In [8]:
import google.generativeai as genai

# Configurar el modelo de Gemini
model = genai.GenerativeModel('gemini-1.5-pro')  # Usa el modelo que prefieras

Probamos el modelo haciendo una pregunta sencilla

In [9]:
# Probar el modelo con una pregunta sencilla
pregunta_sencilla = "¿Cuál es la capital de Francia?"
respuesta = model.generate_content(pregunta_sencilla)

# Imprimir la respuesta
print(respuesta.text)

La capital de Francia es París.



El resultado del modelo es una instancia de `AIMessage` que contiene la respuesta. Podemos extraer esta respuesta encadenando el modelo con un analizador de salida [outputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Así es como se ve el encadenamiento del modelo con un analizador de salida:

<img src='images/chain1.png' width="1200">

Para este ejemplo, utilizaremos un `StrOutputParser` simple para extraer la respuesta como una cadena.

In [10]:
from langchain_core.output_parsers import StrOutputParser

def gemini_invoke(input_text):
    # Verificar si input_text es un diccionario de LangChain
    if isinstance(input_text, dict) and "messages" in input_text:
        # Extraer el contenido del mensaje del usuario
        messages = input_text["messages"]
        if messages and hasattr(messages[0], "content"):
            input_text = messages[0].content

    # Asegurar que el input sea string
    if not isinstance(input_text, str):
        input_text = str(input_text)

    response = model.generate_content(input_text)  
    return response.text


# Creamos un analizador de salida (StrOutputParser) para asegurarnos de que la respuesta sea una cadena
parser = StrOutputParser()

# Creamos la cadena combinando el modelo y el parser
chain = gemini_invoke | parser

# Probamos la cadena con una pregunta sencilla
pregunta_sencilla = "¿Cuál es la capital de Alemania?"
respuesta_parseada = chain.invoke(pregunta_sencilla)

# Imprimimos la respuesta (ahora debería ser una cadena directamente)
print("Respuesta:", respuesta_parseada)
print("Tipo de respuesta:", type(respuesta_parseada))  # Para verificar que es un string

Respuesta: La capital de Alemania es Berlín.

Tipo de respuesta: <class 'str'>


## Presentamos las plantillas de preguntas

Queremos contextualizar el modelo y la pregunta. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) Son una forma sencilla de definir y reutilizar indicaciones.

In [11]:
from langchain.prompts import ChatPromptTemplate

# Definir la plantilla de pregunta
template = """
Responda la pregunta según el contexto descrito a continuación. Si no puede responder, responda "No lo sé".

Contexto: {contexto}

Pregunta: {pregunta}
"""

# Crear la plantilla de prompt
prompt = ChatPromptTemplate.from_template(template)

# Probar la plantilla con un ejemplo
contexto_ejemplo = "París es la capital de Francia y una de las ciudades más visitadas del mundo."
pregunta_ejemplo = "¿Cuál es la capital de Francia?"

# Formatear el prompt con los valores de contexto y pregunta
formatted_prompt = prompt.format(contexto=contexto_ejemplo, pregunta=pregunta_ejemplo)

# Usar el modelo de Gemini para generar una respuesta
respuesta = model.generate_content(formatted_prompt)

# Imprimir la respuesta
print("Respuesta:", respuesta.text)

Respuesta: París



Ahora podemos encadenar el mensaje con el modelo y el analizador de salida.

<img src='images/chain2.png' width="1200">

In [12]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

chain = (
    RunnableParallel(
        contexto=RunnablePassthrough(),
        pregunta=RunnablePassthrough()
    )
    | prompt
    | RunnableLambda(gemini_invoke)
    | parser
)


## Combinación de cadenas

Podemos combinar diferentes cadenas para crear flujos de trabajo más complejos. Por ejemplo, creemos una segunda cadena que traduzca la respuesta de la primera a otro idioma.

Comencemos creando una nueva plantilla de solicitud para la cadena de traducción:

In [13]:
translation_prompt = ChatPromptTemplate.from_template(
    "Traduce {answer} al {language}"
)


Ahora podemos crear una nueva cadena de traducción que combine el resultado de la primera cadena con la solicitud de traducción.

Así es como se ve el nuevo flujo de trabajo:

<img src='images/chain3.png' width="1200">

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

# Plantilla para asegurar respuestas directas y concisas
qa_template = ChatPromptTemplate.from_template("""
Responde únicamente con la respuesta correcta basada en el contexto.

Contexto: {contexto}

Pregunta: {pregunta}

Devuelve solo la respuesta sin explicaciones ni comentarios adicionales.
""")

# Función de invocación con limpieza
def clean_gemini_invoke(inputs):
    response = gemini_invoke(inputs)  # Llamamos a Gemini
    if isinstance(response, dict) and "text" in response:
        return response["text"].strip()
    return response.strip()

# Cadena de QA
qa_chain = (
    {
        "contexto": RunnablePassthrough(),
        "pregunta": RunnablePassthrough()
    }
    | qa_template
    | RunnableLambda(clean_gemini_invoke)
)

# Plantilla de traducción simplificada
translation_template = ChatPromptTemplate.from_template("""
Traduce al {language} el siguiente texto sin modificarlo:

{text}
Solo devuelve el texto traducido, sin explicaciones ni estructura JSON.
""")

# Cadena de traducción asegurando solo texto
translation_chain = (
    {
        "text": RunnablePassthrough(),
        "language": lambda _: "Castellano"
    }
    | translation_template
    | RunnableLambda(clean_gemini_invoke)
)

# Función que encadena QA y traducción
def qa_then_translate(inputs):
    respuesta_qa = qa_chain.invoke(inputs)  # Obtenemos respuesta en inglés
    return translation_chain.invoke({"text": respuesta_qa})  # Traducimos solo el texto

# Cadena combinada con correcciones
combined_chain = RunnableLambda(qa_then_translate)

# Prueba final
inputs = {
    "contexto": "París es la capital de Francia y una de las ciudades más visitadas del mundo.",
    "pregunta": "¿Cuál es la capital de Francia?"
}

respuesta_traducida = combined_chain.invoke(inputs)

print(respuesta_traducida)  # Eliminamos "Respuesta traducida:"


París


# PARTE II

## Transcripcion de video de YouTube

El contexto que queremos enviar al modelo proviene de un video de YouTube. Descargamos el video y transcribámoslo con [OpenAI's Whisper](https://openai.com/research/whisper).

In [ ]:
import tempfile
import whisper
import os
import yt_dlp  # Using yt-dlp instead of pytube

YOUTUBE_VIDEO = 'https://www.youtube.com/watch?v=qcJM0bM3D0Q'  # Coloca tu enlace de YouTube aquí

if not os.path.exists("transcription.txt"):
    print(f"Descargando video: {YOUTUBE_VIDEO}")
    
    # Crear un directorio temporal para la descarga
    with tempfile.TemporaryDirectory() as tmpdir:
        # Opciones de yt-dlp para descargar solo el audio
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(tmpdir, 'audio.%(ext)s'),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False
        }
        
        # Descargar el audio
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.extract_info(YOUTUBE_VIDEO, download=True)
            audio_file = os.path.join(tmpdir, 'audio.mp3')
        
        print(f"Transcribiendo archivo de audio: {audio_file}")
        
        # Cargar el modelo Whisper
        whisper_model = whisper.load_model("base")  # Puedes usar "small", "medium", o "large" si prefieres más precisión
        
        # Transcribir el audio
        transcription = whisper_model.transcribe(audio_file, fp16=False)["text"].strip()
        
        # Guardar la transcripción en un archivo
        with open("transcription.txt", "w") as file:
            file.write(transcription)
        
        print("Transcripción completada y guardada en 'transcription.txt'")
else:
    print("¡El archivo de transcripción ya existe!")


¡El archivo de transcripción ya existe!


Vamos a leer la transcripción y mostrar los primeros caracteres para asegurarnos de que todo funciona como se espera.

In [18]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

"Every time I think of you I feel shot right through with a ball of blue It's no problem to mine but "

## Usando la transcripción completa como contexto

Si intentamos invocar la cadena usando la transcripción como contexto, el modelo devolverá un error porque el contexto es demasiado largo.

Los modelos de lenguaje grandes admiten tamaños de contexto limitados. El vídeo que estamos usando es demasiado largo para que el modelo lo pueda procesar, por lo que necesitamos buscar una solución diferente.

In [20]:
# Primero, aseguramos que tengamos la transcripción cargada
with open("transcription.txt", "r") as file:
    transcription = file.read()


# Ahora intentamos usar la transcripción como contexto
try:
    prompt = f"""
    Contexto: {transcription}
    Pregunta: ¿De queé habla esta canción?
    """

    # Realizamos la solicitud al modelo de Gemini
    response = genai.GenerativeModel('gemini-1.5-pro').generate_content(prompt)
    
    # Imprimir la respuesta
    print("Respuesta de Gemini:", response.text)

except Exception as e:
    print("Error:", e)


Respuesta de Gemini: Esta canción habla de la confusión y la frustración del amor no correspondido o de una relación difícil. El narrador está claramente afectado por alguien ("Cada vez que pienso en ti me siento atravesado por una bola azul") pero no puede articular sus sentimientos ("Ves las palabras que no puedo decir"). 

Hay una sensación de anhelo y nostalgia por un tiempo más simple ("¿Por qué no podemos ser nosotros mismos como ayer?"). El narrador se siente solo y perdido a pesar de intentar mantener una apariencia de estar bien ("Estoy bastante simple, he estado solo", "Me siento bien y me siento bien").

La repetición de "Cada vez que veo caer" junto con la sensación de soledad ("He estado solo") sugiere una vulnerabilidad y una sensación de estar perdiendo el control o de "caer" en el amor o en la desesperación.  La "sabiduría del tonto" que no liberará al narrador podría referirse a consejos bienintencionados pero inútiles o a la propia incapacidad del narrador para razona

## División de la transcripción

Dado que no podemos usar la transcripción completa como contexto para el modelo, una posible solución es dividir la transcripción en fragmentos más pequeños. Así, podemos invocar el modelo utilizando solo los fragmentos relevantes para responder a una pregunta específica:

<img src='images/system2.png' width="1200">

Comencemos cargando la transcripción en la memoria:

In [21]:
from langchain_community.document_loaders import TextLoader

# Load the transcription file
loader = TextLoader("transcription.txt")
documents = loader.load()

# Print basic info about the loaded document
print(f"Loaded {len(documents)} document")
print(f"Text length: {len(documents[0].page_content)} characters")

Loaded 1 document
Text length: 1213 characters


Hay muchas maneras de dividir un documento. En este ejemplo, usaremos un divisor simple que divide el documento en fragmentos de tamaño fijo. Consulta [Divisores de texto](https://python.langchain.com/docs/modules/data_connection/document_transformers/) para obtener más información sobre los diferentes enfoques para dividir documentos.

A modo de ejemplo, dividiremos la transcripción en fragmentos de 100 caracteres con una superposición de 20 caracteres y mostraremos los primeros fragmentos:

In [22]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# First load the document
loader = TextLoader("transcription.txt")
documents = loader.load()

# Create a text splitter with chunk size of 100 and overlap of 20 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

# Split the document into chunks
chunks = text_splitter.split_documents(documents)

# Display information about the chunks
print(f"Split the document into {len(chunks)} chunks")

# Show the first 3 chunks as an example
print("\nFirst three chunks:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}:")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Content: {chunk.page_content}")

Split the document into 15 chunks

First three chunks:

Chunk 1:
Length: 99 characters
Content: Every time I think of you I feel shot right through with a ball of blue It's no problem to mine but

Chunk 2:
Length: 98 characters
Content: problem to mine but it's a problem to find Living alive that I can't be behind There's no sense in

Chunk 3:
Length: 96 characters
Content: There's no sense in telling me The wisdom of the fool won't set you free But that's the way that


Para nuestra aplicación específica, utilizaremos 1000 caracteres en su lugar:

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter with chunk size of 1000 and overlap of 200 characters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Split the document into chunks
chunks = text_splitter.split_documents(documents)

# Display information about the chunks
print(f"Split the document into {len(chunks)} chunks")

# Show the first chunk as an example
if chunks:
    print("\nFirst chunk:")
    print(f"Length: {len(chunks[0].page_content)} characters")
    print(f"Content: {chunks[0].page_content}")

Split the document into 2 chunks

First chunk:
Length: 998 characters
Content: Every time I think of you I feel shot right through with a ball of blue It's no problem to mine but it's a problem to find Living alive that I can't be behind There's no sense in telling me The wisdom of the fool won't set you free But that's the way that goes and it's what nobody knows Well, every day my confusion grows Every time I see all that I get out of my knees and breathe I'm pretty simple, I've been on my own You see the words I can't say I feel fine and I feel good I feel like I'm never sure Wherever I get this way I just don't know what to say Why can't we be ourselves like we were yesterday I'm not sure what this could mean I don't think you'll watch you say I do admit to myself that if I had someone else Then I'll never see it just what I'm not to be Every time I see falling I've been on my own You see the words I can't say Every time I see falling I've been on my own You see the words I can't s

# PARTE III

## Configuración de un Vector Store

Necesitamos una forma eficiente de almacenar fragmentos de documentos, sus Embeddings y realizar búsquedas de similitud a gran escala. Para ello, usaremos un Vector Store.

Un Vector Store es una base de datos de Embeddings especializada en búsquedas rápidas de similitud.


<img src='images/chain4.png' width="1200">

Necesitamos configurar un retriever (https://python.langchain.com/docs/how_to/#retrievers). Este retriever realizará una búsqueda de similitud en el almacén vectorial y devolverá los documentos más similares al siguiente paso de la cadena.

## Configurar Pinecone

Para este ejemplo, usaremos [Pinecone](https://www.pinecone.io/).

<img src="images/pinecone.png" width="800">

El primer paso es crear una cuenta de Pinecone, configurar un índice, obtener una clave API y configurarla como variable de entorno `PINECONE_API_KEY`.

In [24]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Create Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create the index name 
index_name = "rag-transcription"

# Si el índice existe, lo eliminamos primero para recrearlo con la dimensión correcta
if index_name in pc.list_indexes().names():
    print(f"Eliminating existing index '{index_name}' to recreate with correct dimensions")
    pc.delete_index(index_name)
    # Esperar un momento para que la eliminación se complete
    import time
    time.sleep(5)

# Create the index with the correct dimensions
pc.create_index(
    name=index_name,
    dimension=384,  # HuggingFace 'all-MiniLM-L6-v2' embeddings have 384 dimensions
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
print(f"Created new index '{index_name}' with dimension 384")

# Initialize HuggingFace embeddings model (no API key needed, runs locally)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store and load the documents
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,  # Use your previously created chunks
    embedding=embeddings,
    index_name=index_name
)

print(f"Successfully loaded {len(chunks)} chunks into Pinecone index '{index_name}'")

Created new index 'rag-transcription' with dimension 384


/tmp/ipykernel_71694/1861014624.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Successfully loaded 2 chunks into Pinecone index 'rag-transcription'


Ahora ejecutemos una búsqueda de similitud en pinecone para asegurarnos de que todo funciona:

In [25]:
# Test similarity search
query = "What are the main topics discussed in the video?"
docs = vectorstore.similarity_search(query, k=3)

print("\nResults for query:", query)
print("-" * 50)
for i, doc in enumerate(docs, 1):
    print(f"\nResult {i}:")
    print(doc.page_content)
    print("-" * 50)


Results for query: What are the main topics discussed in the video?
--------------------------------------------------

Result 1:
I've been on my own You see the words I can't say Every time I see falling I've been on my own You see the words I can't say Every time I see falling I've been on my own Every time I see falling I've been on my own You see the words I can't say Every time I see falling I've been on my own You see the words I can't say I do admit to myself that if I had someone else Then I'll never see it just what I'm not to be
--------------------------------------------------

Result 2:
Every time I think of you I feel shot right through with a ball of blue It's no problem to mine but it's a problem to find Living alive that I can't be behind There's no sense in telling me The wisdom of the fool won't set you free But that's the way that goes and it's what nobody knows Well, every day my confusion grows Every time I see all that I get out of my knees and breathe I'm prett

Configuremos la nueva cadena usando Pinecone como almacén vectorial:

In [28]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Crear el prompt de Gemini
template = """
Responde a la siguiente pregunta basándote en el contexto proporcionado:

Contexto:
{context}

Pregunta:
{question}

Responde según el contexto proporcionado. Si no puedes encontrar la respuesta, di "No lo sé".
"""

def create_prompt(context, question):
    return template.format(context=context, question=question)

# Procesar la cadena RAG con Gemini
def rag_chain(question):
    # Realizar la búsqueda de similitud
    relevant_docs = similarity_search(question, k=3)

    # Concatenar los fragmentos relevantes para formar el contexto
    context = "\n".join([doc.page_content for doc in relevant_docs])

    # Crear el prompt
    prompt = create_prompt(context, question)

    try:
        # Generar la respuesta usando Gemini
        response = genai.GenerativeModel('gemini-1.5-pro').generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error en la generación: {e}")
        return None

# Probar la cadena RAG
question = "What are the key points discussed in the video?"
response = rag_chain(question)

# Mostrar la respuesta
print("\nPregunta:", question)
print("\nRespuesta:", response)



Pregunta: What are the key points discussed in the video?

Respuesta: The provided text seems to be song lyrics, not a video.  The key themes are:

* **Intense feelings for someone:**  "Every time I think of you I feel shot right through with a ball of blue" suggests a powerful, possibly painful emotional reaction.
* **Confusion and uncertainty:**  "Every day my confusion grows," "I feel like I'm never sure," and "I just don't know what to say" express a lack of clarity in the relationship and the speaker's own feelings.
* **Longing for connection but struggling to express it:** The lyrics hint at difficulty communicating ("You see the words I can't say") and a desire for authenticity ("Why can't we be ourselves like we were yesterday").
* **Solitude and independence:** The repeated phrase "I've been on my own" emphasizes the speaker's solitary state.
* **Ambivalence about being alone:** While acknowledging their independence, the speaker also admits "if I had someone else Then I'll n